In [1]:

import tensorflow as tf

In [2]:
#setup log 
%load_ext tensorboard
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(log_dir)

2024-10-16 17:44:29.775156: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-10-16 17:44:29.775171: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-10-16 17:44:29.775183: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-10-16 17:44:29.775207: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-16 17:44:29.775219: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
import tensorflow_io as tfio
import librosa as lr

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import config as cfg
import os
SCRIPT_DIR = os.path.abspath('')
classifier = tf.keras.models.load_model(os.path.join(SCRIPT_DIR, cfg.MODEL_PATH), compile=True)

In [6]:
ds = tf.data.Dataset.list_files("example/vowa/*.flac")

In [7]:
@tf.py_function(Tout=tf.float32)
def decode_flac(filename):
    audio, sr = lr.load(filename.numpy(), sr=None)
    assert sr==48000
    return (tf.convert_to_tensor(audio), filename)

def create_slices(audio, filename):
    snippet_length = 48000*3
    padding = tf.zeros((snippet_length - tf.shape(audio)[0] % snippet_length),dtype=tf.float32)
    padded = tf.concat([audio,padding],0)
    chunks = tf.reshape(padded,(-1, 144000))
    return tf.data.Dataset.from_tensor_slices((chunks, filename))

ds = ds.map(decode_flac,num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.interleave(create_slices, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [8]:
def execute(batches):
    logits = []
    for chunk, filename in batches:
        logits.append(classifier.basic(chunk)['scores'])
    return logits
y = execute(ds.batch(100, drop_remainder=False))

2024-10-16 17:44:40.632124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [10]:
383*100*3/3600

31.916666666666668

In [13]:
acc = 0
ds = tf.data.Dataset.list_files("example/vowa/*.flac")
for file in ds:
    audio, sr = lr.load(file.numpy(), sr=None)
    assert sr==48000
    duration = lr.get_duration(y=audio, sr=sr)
    print(f"{file.numpy()} has duration {duration}")
    acc = acc + duration
    

b'example/vowa/SMA15090_20240514_170222.flac' has duration 1.4933333333333334
b'example/vowa/SMA15090_20240515_075202.flac' has duration 3598.0
b'example/vowa/SMA15090_20240514_174957.flac' has duration 3542.9973333333332
b'example/vowa/SMA15090_20240515_115202.flac' has duration 3598.0
b'example/vowa/SMA15090_20240515_175202.flac' has duration 3598.0
b'example/vowa/SMA15090_20240516_045100.flac' has duration 3600.0
b'example/vowa/SMA15090_20240516_075102.flac' has duration 3598.0
b'example/vowa/SMA15090_20240515_195202.flac' has duration 3598.0
b'example/vowa/SMA15090_20240514_214902.flac' has duration 298.0
b'example/vowa/SMA15090_20240515_105202.flac' has duration 3598.0
b'example/vowa/SMA15090_20240516_095102.flac' has duration 3598.0
b'example/vowa/SMA15090_20240516_115102.flac' has duration 3598.0
b'example/vowa/SMA15090_20240516_105102.flac' has duration 3598.0
b'example/vowa/SMA15090_20240515_215202.flac' has duration 178.0
b'example/vowa/SMA15090_20240516_065102.flac' has dura

In [15]:
acc/3600

31.886503703703703